# Lung Cancer Prediction

We are using a dataset of three different kinds of images which the system will learn from 

In [4]:
from torch.utils.data import Dataset, DataLoader
import torch
from torchvision import transforms
import cv2
from sklearn.model_selection import train_test_split
from torch.nn import BCELoss
from tqdm import tqdm
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

C:\Users\ihsan\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import numpy as np
import glob
import matplotlib as plt
from PIL import Image
from torch.utils.tensorboard import SummaryWriter
import logging

In [6]:
!nvidia-smi

Mon Dec  4 21:25:11 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.92                 Driver Version: 545.92       CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1050 Ti   WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A    0C    P0              N/A / ERR! |      0MiB /  4096MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [7]:
class LData(Dataset):
    def __init__(self, path, trans = None):
        self.location = path
        self.lgdata = [] #save data here
        self.labels = []# save each corresponding labels here
        self.lb = []
        self.trans = trans
        
    def __getitem__(self, serial):
        i=0
        for x in glob.glob(self.location+"\*"):
            self.labels.append(x)
            for y in glob.glob(self.labels[i]+"\*"):
                self.lgdata.append(y)
                self.lb.append(i)
            i+=1
        img = Image.open(self.lgdata[serial]).convert('L') # convert('RGB') to colored
        if self.trans is not None:
            img = self.trans(img)   
        return img, self.lb[serial]
    def __len__(self):
        return len(self.lb)
        
        

In [8]:
def BCE(labels, outputs):
    loss = torch.nn.CrossEntropyLoss()
#     outputs = outputs.squeeze(1)
    loss = loss(outputs, labels)
    return loss

def KLdiv(labels, outputs):
    loss = KLdiv()
    loss = loss(outputs, labels)
    return loss
def KD(labels, outputs, teacher_outputs):
    alpha = 0.8
    T = 20

    loss_CE = F.cross_entropy(outputs, labels)
    D_KL = nn.KLDivLoss()(F.log_softmax(outputs/T, dim=1),
                          F.softmax(teacher_outputs/T, dim=1)) * (T * T)
    KD_loss = (1. - alpha)*loss_CE + alpha*D_KL

    return KD_loss


In [10]:
def evaluate(model, valset, n = True):
    correct = 0
    total = 0
    data_sc = []
    data_rc = []
    model.eval()
    print("-"*44, "Evaluation", "-"*44)

    with tqdm(total=len(valset), colour = 'green') as t:
        for _, (inputs, labels) in enumerate(valset):
            if torch.cuda.is_available():
                inputs = inputs.to('cuda')
                labels = labels.to('cuda')
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            
            predicted = predicted.cpu().data.numpy()
            label_1 = labels.clone().detach()
            label_1 = label_1.cpu().data.numpy()
            data_sc.extend(predicted)
            data_rc.extend(label_1)
#             print("Predicted: ", predicted)
#             print("Labels: ", label_1)
            t.update()
        ta = 100 * correct/total
        tl= (100-ta)/100
        ############Precision, Recall, Kappa, F1 Score, Classification Report, and Confusion Matrix##################
        if n == False:
            fscore = f1_score(data_rc, data_sc, average = 'weighted')
            precision = precision_score(data_rc, data_sc, average = 'weighted')
            recall = recall_score(data_rc, data_sc, average = 'weighted')
            kappa = cohen_kappa_score(data_rc, data_sc)
            print("f1_score: ", fscore)
            print("recall_score: ", recall)
            print("cohen_kappa_score: ", kappa)
            print("precision: ", precision)
            print(classification_report(data_rc, data_sc))
            print("Confusion Matrix: ", confusion_matrix(data_rc, data_sc))
        return ta, tl


In [11]:
import time

def train(model, trainset, valset, optimizer, criterion, epochs):
    best_accuracy = 0
    correct = 0
    total =0
    y = 0
    logging.basicConfig(filename='train_1.log', encoding='utf-8')
    total_starttime = time.time()
    log_dir = '.' + '/tensorboard/'
    writer = SummaryWriter(log_dir=log_dir)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[3, 20, 30], gamma=0.2)
    for i in range(0, epochs):
        loss = 0
        print('-'*100)
        print("Epochs ",i, " out of ", epochs)
        print('-'*100)
        epochstart = time.time()
        scheduler.step(i)
        model.train()
        print("-"*45, "Training", "-"*45)
        with tqdm(total=len(trainset), colour = 'red') as t:
            for _, (inputs, labels) in enumerate(trainset):
                if torch.cuda.is_available():
                    inputs = inputs.to('cuda')
                    labels = labels.to('cuda')
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(labels, outputs)
                loss.backward()
                optimizer.step()                
                _, predicted = outputs.max(1)
                total += labels.size(0)
                correct += predicted.eq(labels).sum().item()
                
                t.set_postfix(loss='{:05.3f}'.format(loss), lr='{:05.6f}'.format(optimizer.param_groups[0]['lr']))
                t.update()
            epochend = time.time()        
        #----------------------Check Again---------------------------#
        train_accuracy = 100 * correct/total
        train_loss= (100-train_accuracy)/100
        #----------------------Check Again---------------------------#
        model.eval()
        del inputs
        del labels
        del outputs
        test_accuracy, test_loss = evaluate(model, valset)
        print("Train Accuracy: ", train_accuracy, "Train Loss: ", train_loss)
        print("Test Accuracy: ", test_accuracy, "Test Loss: ", test_loss)
        if test_accuracy >= best_accuracy:
            print('*'*10, "Congrats!!!! Higher Performance Achieved", '*'*10)
            torch.save({
            'state_dict': model.state_dict(),
            'optimizer' : optimizer.state_dict(),
            }, 'our_model.pth.tar')
            best_accuracy = test_accuracy
        
        writer.add_scalar('Loss/train', train_loss,  i)
        writer.add_scalar('Loss/test', test_loss, i)
        writer.add_scalar('Accuracy/train', train_accuracy, i)
        writer.add_scalar('Accuracy/test', test_accuracy, i)
        logging.info("- Train accuracy: {acc:.4f}, training loss: {loss:.4f}".format(acc = train_accuracy, loss = train_loss))
        logging.info("- Evaluation accuracy: {acc:.4f}, Evaluation loss: {loss:.4f}".format(acc = test_accuracy, loss = test_loss))

            
    total_endtime = time.time()
    total_endtime = total_endtime - total_starttime
    tm = time.gmtime(total_endtime)
    print("Time taken to complete training: ", tm.tm_mon," months", tm.tm_mday, " days", 
                    tm.tm_hour, " hours", tm.tm_min, " minutes", tm.tm_sec, "seconds")

In [12]:
def train_kd(model, teacher_model, train_loader, valid_loader, optimizer, criterion, epochs):
    
    best_accuracy = 0
    correct = 0
    total =0
    y = 0
    total_starttime = time.time()
    log_dir = '.' + '/tensorboard/'
    writer = SummaryWriter(log_dir=log_dir)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[3, 20, 30], gamma=0.2)
    for i in range(0, epochs):
        loss = 0
        print('-'*100)
        print("Epochs ",i, " out of ", epochs)
        print('-'*100)
        epochstart = time.time()
        scheduler.step(i)
        model.train()
        teacher_model.eval()
        print("-"*45, "Training", "-"*45)
        with tqdm(total=len(trainset), colour = 'red') as t:
            for _, (inputs, labels) in enumerate(trainset):
                if torch.cuda.is_available():
                    inputs = inputs.to('cuda')
                    labels = labels.to('cuda')
                optimizer.zero_grad()
                outputs = model(inputs)
                teacher_outputs = teacher_model(inputs)
                loss = criterion(labels, outputs, teacher_outputs)
                loss.backward()
                optimizer.step()                
                _, predicted = outputs.max(1)
                total += labels.size(0)
                correct += predicted.eq(labels).sum().item()
                
                t.set_postfix(loss='{:05.3f}'.format(loss), lr='{:05.6f}'.format(optimizer.param_groups[0]['lr']))
                t.update()
            epochend = time.time()        
        #----------------------Check Again---------------------------#
        train_accuracy = 100 * correct/total
        train_loss= (100-train_accuracy)/100
        #----------------------Check Again---------------------------#
        model.eval()
        del inputs
        del labels
        del outputs
        test_accuracy, test_loss = evaluate(model, valset)
        print("Train Accuracy: ", train_accuracy, "Train Loss: ", train_loss)
        print("Test Accuracy: ", test_accuracy, "Test Loss: ", test_loss)
        if test_accuracy >= best_accuracy:
            print('*'*10, "Congrats!!!! Higher Performance Achieved", '*'*10)
            torch.save({
            'state_dict': model.state_dict(),
            'optimizer' : optimizer.state_dict(),
            }, 'testing.pth.tar')
            best_accuracy = test_accuracy
        
        writer.add_scalar('Loss/train', train_loss,  i)
        writer.add_scalar('Loss/test', test_loss, i)
        writer.add_scalar('Accuracy/train', train_accuracy, i)
        writer.add_scalar('Accuracy/test', test_accuracy, i)
        logging.info("- Train accuracy: {acc:.4f}, training loss: {loss:.4f}".format(acc = train_accuracy, loss = train_loss))
        logging.info("- Evaluation accuracy: {acc:.4f}, Evaluation loss: {loss:.4f}".format(acc = test_accuracy, loss = test_loss))

            
    total_endtime = time.time()
    total_endtime = total_endtime - total_starttime
    tm = time.gmtime(total_endtime)
    print("Time taken to complete training: ", tm.tm_hour, " hours", tm.tm_min, " minutes", tm.tm_sec, "seconds")
    

In [13]:
#Biulding our own model
import torch.nn as nn

class OwnModel(nn.Module):
    def __init__(self, num_classes=10):
        super(OwnModel, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 128, kernel_size=3, stride=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 64, kernel_size=3),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 8, kernel_size=5),
            nn.ReLU(inplace=True),
#             nn.Conv2d(32, 16, kernel_size=3),
#             nn.ReLU(inplace=True),
            nn.Dropout(p=0.25)
#             nn.Conv2d(200, 100, kernel_size=3, padding=1),
#             nn.ReLU(inplace=True),
#             nn.Conv2d(100, 150, kernel_size=3, padding=1),
#             nn.ReLU(inplace=True),
#             nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Linear(3872, 500),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(500, 250),
            nn.ReLU(inplace=True),
            nn.Linear(250, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
#         x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [14]:
# ##### INFERENCE TIME#################
# def run_benchmark(model_path: str, shape: Optional[List[int]] = None, verbose: bool = True) -> float:
#     command = f"benchmark_app -m {model_path} -d CPU -api async -t 15"
#     if shape is not None:
#         command += f' -shape [{",".join(str(x) for x in shape)}]'
#     cmd_output = subprocess.check_output(command, shell=True)  # nosec
#     if verbose:
#         print(*str(cmd_output).split("\\n")[-9:-1], sep="\n")
#     match = re.search(r"Throughput\: (.+?) FPS", str(cmd_output))
#     return float(match.group(1))

##### MODEL SIZE CALCULATOR#########
def get_model_size(ir_path: str, m_type: str = "Mb", verbose: bool = True) -> float:
    xml_size = os.path.getsize(ir_path)
    bin_size = os.path.getsize(os.path.splitext(ir_path)[0] + ".bin")
    for t in ["bytes", "Kb", "Mb"]:
        if m_type == t:
            break
        xml_size /= 1024
        bin_size /= 1024
    model_size = xml_size + bin_size
    if verbose:
        print(f"Model graph (xml):   {xml_size:.3f} Mb")
        print(f"Model weights (bin): {bin_size:.3f} Mb")
        print(f"Model size:          {model_size:.3f} Mb")
    return model_size

def model_size_real(model):
    param_size = 0
    for param in model.parameters():
        param_size += param.nelement() * param.element_size()
    buffer_size = 0
    for buffer in model.buffers():
        buffer_size += buffer.nelement() * buffer.element_size()

    size_all_mb = (param_size + buffer_size) / 1024**2
    print('model size: {:.3f}MB'.format(size_all_mb))

In [15]:
def measure_inference_latency(model,
                              device,
                              input_size=(10, 1, 224, 224),
                              num_samples=100,
                              num_warmups=10):

    model.to(device)
    model.eval()

    x = torch.rand(size=input_size).to(device)

    with torch.no_grad():
        for _ in range(num_warmups):
            _ = model(x)
    torch.cuda.synchronize()

    with torch.no_grad():
        start_time = time.time()
        for _ in range(num_samples):
            _ = model(x)
            torch.cuda.synchronize()
        end_time = time.time()
    elapsed_time = end_time - start_time
    elapsed_time_ave = elapsed_time / num_samples

    return elapsed_time_ave

In [13]:
import model.alexnet as alexnet
import model.resnet as resnet
import model.mobilenetv2 as mobilenet
import model.shufflenetv2 as shufflenet

def main():
    system_control = 0
    #0 Normal training
    #1 Evaluation
    #2 KD Train
    #3 Quantization
    
    model = "OwnModel"
    criterion = "CE"
    dataset = "Base"
    if model == "AlexNet":
        model = alexnet.AlexNet(num_classes = 3).to('cuda')
        batch_size = 64
    elif model == "ResNet18":
        model = resnet.ResNet18(num_classes = 3).to('cuda')
        batch_size = 5
    elif model == "ShuffleNetV2":
        model = shufflnet.shufflenetv2(class_num = 3).to('cuda')
        batch_size = 10
    elif model == "ResNet50":
        model = resnet.ResNet50(num_classes = 3).to('cuda')
        batch_size = 2
    elif model == "ResNet152":
        model = resnet.ResNet152(num_classes = 3).to('cuda')
        batch_size = 5
    elif model == "MobileNetV2":
        model = mobilenet.mobilenetv2(class_num = 3).to('cuda')
        batch_size = 5
    elif model == "OwnModel":
        model = OwnModel(num_classes = 10).to('cuda')
        batch_size = 10 
    if criterion == "CE":
        criterion = BCE
    elif criterion == "KLD":
        criterion = KLD
    elif criterion == "KD":
        criterion = KD
    else:
        criterion = ''
    epochs = 50
    optimizer = torch.optim.Adam(params = model.parameters(), lr = 0.001, capturable = True)#backpropagating
    if dataset == "Base": 
        path=".\L_dataset"
        transform = transforms.Compose([
                transforms.Resize((224,224)),
                transforms.ToTensor(),
            #         transforms.Normalize(0.2859, 0.3530)
        ])
        LD = LData(path, transform)
        LD[0]
        print("Total Images in the Dataset: ", len(LD))
        complete_set = torch.utils.data.DataLoader(LD, batch_size=1096, num_workers=0, shuffle=True)
        train_idx = valid_idx = 0
        image,labels = next(iter(complete_set))
        labels = labels.to('cpu').numpy()
        print("Labels Visual: ", labels)
        #stratified train-test-split to get same percentage from all the classes
        train_idx, valid_idx = train_test_split(np.arange(len(labels)), test_size=0.2,           #scikit learn train test split
                                                random_state=42, shuffle=True, stratify=labels)  #for class imbalance issue
        print("Train Set Size: ", len(train_idx))
        print("Validation Set Size: ", len(valid_idx))
        train_sampler = torch.utils.data.SubsetRandomSampler(train_idx)
        valid_sampler = torch.utils.data.SubsetRandomSampler(valid_idx)
        train_loader = torch.utils.data.DataLoader(LD, batch_size=batch_size, sampler=train_sampler)
        valid_loader = torch.utils.data.DataLoader(LD, batch_size=batch_size, sampler=valid_sampler)
        
        ######Only Evaluate a model#######
        if system_control == 1:
            model = alexnet.AlexNet(num_classes = 3).to('cuda')
            checkpoint = torch.load("99.54.path.tar", map_location=lambda storage, loc: storage)
            model.load_state_dict(checkpoint['state_dict'])        
            model.eval()
            ta,tl = evaluate(model, valid_loader, False)
            print(ta)
        
        model_size_real(model)
        print(measure_inference_latency(model, 'cuda'))
        #TRAIN A MODEL
        if system_control == 0:
            model_size_real(model)
            train(model, train_loader, valid_loader, optimizer, criterion, epochs)
        
        #TRAIN KD MODEL
        if system_control == 2:
            teacher_model = ResNet18
            if teacher_model == "AlexNet":
                teacher_model = alexnet.AlexNet(num_classes = 3).to('cuda')
                checkpoint = torch.load("AlexNet_train_1.pth.tar", map_location=lambda storage, loc: storage)
                teacher_model.load_state_dict(checkpoint['state_dict'], strict = False)
                batch_size = 64
            elif teacher_model == "ResNet18":
                teacher_model = resnet.ResNet18(num_classes = 3).to('cuda')
                checkpoint = torch.load("AlexNet_train_1.pth.tar", map_location=lambda storage, loc: storage)
                teacher_model.load_state_dict(checkpoint['state_dict'], strict = False)
                batch_size = 5
            elif teacher_model == "ShuffleNetV2":
                teacher_model = shufflnet.shufflenetv2(class_num = 3).to('cuda')
                checkpoint = torch.load("AlexNet_train_1.pth.tar", map_location=lambda storage, loc: storage)
                teacher_model.load_state_dict(checkpoint['state_dict'], strict = False)
                batch_size = 10
            elif teacher_model == "ResNet50":
                teacher_model = resnet.ResNet50(num_classes = 3).to('cuda')
                checkpoint = torch.load("AlexNet_train_1.pth.tar", map_location=lambda storage, loc: storage)
                teacher_model.load_state_dict(checkpoint['state_dict'], strict = False)
                batch_size = 2
            elif teacher_model == "ResNet152":
                teacher_model = resnet.ResNet152(num_classes = 3).to('cuda')
                checkpoint = torch.load("AlexNet_train_1.pth.tar", map_location=lambda storage, loc: storage)
                teacher_model.load_state_dict(checkpoint['state_dict'], strict = False)
                batch_size = 5
            elif teacher_model == "MobileNetV2":
                teacher_model = mobilenet.mobilenetv2(class_num = 3).to('cuda')
                checkpoint = torch.load("AlexNet_train_1.pth.tar", map_location=lambda storage, loc: storage)
                teacher_model.load_state_dict(checkpoint['state_dict'], strict = False)
                batch_size = 5
            elif teacher_model == "OwnModel":
                teacher_model = OwnModel(num_classes = 10).to('cuda')
                checkpoint = torch.load("AlexNet_train_1.pth.tar", map_location=lambda storage, loc: storage)
                teacher_model.load_state_dict(checkpoint['state_dict'], strict = False)
                batch_size = 10 
            train_kd(model, teacher_model, train_loader, valid_loader, optimizer, criterion, epochs)
        
#         #Quantize a MODEL using quantization aware training
        if system_control == 3:
            model = alexnet.AlexNet(num_classes = 3).to('cuda')
            checkpoint = torch.load("99.54.path.tar", map_location=lambda storage, loc: storage)
            model.load_state_dict(checkpoint['state_dict'])        
            model.eval()
            ta,tl = evaluate(model, valid_loader, False)
            print(ta)
        
            model_size_real(model)
            print(measure_inference_latency(model, 'cuda'))
            from neural_compressor import quantization
            from neural_compressor.config import PostTrainingQuantConfig
            print(model)
            conf = PostTrainingQuantConfig(approach = "dynamic") # default approach is "auto", you can set "dynamic":PostTrainingQuantConfig(approach="dynamic")
            q_model = quantization.fit(model=model,
                                       conf=conf)
            print(q_model)
    #         q_model.save('./alex99.54_output')
            torch.save({
                'state_dict': q_model.state_dict(),
                }, 'AlexNet_train_1.pth.tar')
            model = alexnet.AlexNet(num_classes = 3).to('cuda')
            checkpoint = torch.load("AlexNet_train_1.pth.tar", map_location=lambda storage, loc: storage)
            model.load_state_dict(checkpoint['state_dict'], strict = False)
            ta,tl = evaluate(model, valid_loader, False)
            print(ta)
            model_size_real(model)
            print(measure_inference_latency(model, 'cuda'))


In [18]:
import warnings
warnings.filterwarnings('ignore')
import os
if __name__ == '__main__':
    main()

Total Images in the Dataset:  1097
Labels Visual:  [1 2 1 ... 1 2 2]
Train Set Size:  876
Validation Set Size:  220
model size: 8.210MB
0.012027931213378907
model size: 8.210MB
----------------------------------------------------------------------------------------------------
Epochs  0  out of  50
----------------------------------------------------------------------------------------------------
--------------------------------------------- Training ---------------------------------------------


100%|█████████████████████████████████████████████████████████| 88/88 [00:29<00:00,  2.96it/s, loss=1.079, lr=0.001000]


-------------------------------------------- Evaluation --------------------------------------------


100%|██████████████████████████████████████████████████████████████████████████████████| 22/22 [00:06<00:00,  3.53it/s]


Train Accuracy:  45.89041095890411 Train Loss:  0.5410958904109588
Test Accuracy:  49.54545454545455 Test Loss:  0.5045454545454545
********** Congrats!!!! Higher Performance Achieved **********
----------------------------------------------------------------------------------------------------
Epochs  1  out of  50
----------------------------------------------------------------------------------------------------
--------------------------------------------- Training ---------------------------------------------


...

Train Accuracy:  93.02721088435374 Train Loss:  0.06972789115646265
Test Accuracy:  99.0909090909091 Test Loss:  0.009090909090909064
----------------------------------------------------------------------------------------------------
Epochs  49  out of  50
----------------------------------------------------------------------------------------------------
--------------------------------------------- Training ---------------------------------------------


100%|█████████████████████████████████████████████████████████| 88/88 [00:28<00:00,  3.08it/s, loss=0.009, lr=0.000008]


-------------------------------------------- Evaluation --------------------------------------------


100%|██████████████████████████████████████████████████████████████████████████████████| 22/22 [00:05<00:00,  3.75it/s]


Train Accuracy:  93.162100456621 Train Loss:  0.06837899543378995
Test Accuracy:  99.0909090909091 Test Loss:  0.009090909090909064
Time taken to complete training:  1  months 1  days 0  hours 29  minutes 35 seconds


In [16]:
import numpy as np
import gradio as gr
import model.alexnet as alexnet
import model.resnet as resnet
import model.mobilenetv2 as mobilenet
import model.shufflenetv2 as shufflenet

def sepia(input_img):
    transform = transforms.Compose([
                transforms.Resize((224,224)),
                transforms.ToTensor(),
            #         transforms.Normalize(0.2859, 0.3530)
        ])
    img = Image.fromarray(np.uint8(input_img)).convert('L') # convert('RGB') to colored
    img = transform(img)
    img.size()
    img = img.unsqueeze(0)
    img.size()
    img = img.to('cuda')
    model = alexnet.AlexNet(num_classes = 3).to('cuda')
    checkpoint = torch.load("99.54.path.tar", map_location=lambda storage, loc: storage)
    model.load_state_dict(checkpoint['state_dict'], strict = False)
    model.eval()
    outputs = model(img)
    z = torch.argmax(outputs)
    if z == 0:
        s="Benign Case | The tumor has been spotted which is still not harmful."
    elif z == 1:
        s="Malignant Tumor Present | You should immediately consult with a doctor."
    elif z == 2:
        s="Normal Case | No tumor cell has been found, your are safe. "
    return s

demo = gr.Interface(sepia, gr.Image(), "text")
demo.launch() 

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "C:\Users\ihsan\AppData\Local\Programs\Python\Python310\lib\site-packages\gradio\routes.py", line 422, in run_predict
    output = await app.get_blocks().process_api(
  File "C:\Users\ihsan\AppData\Local\Programs\Python\Python310\lib\site-packages\gradio\blocks.py", line 1323, in process_api
    result = await self.call_function(
  File "C:\Users\ihsan\AppData\Local\Programs\Python\Python310\lib\site-packages\gradio\blocks.py", line 1051, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "C:\Users\ihsan\AppData\Local\Programs\Python\Python310\lib\site-packages\anyio\to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "C:\Users\ihsan\AppData\Local\Programs\Python\Python310\lib\site-packages\anyio\_backends\_asyncio.py", line 877, in run_sync_in_worker_thread
    return await future
  File "C:\Users\ihsan\AppData\Local\Programs\Python\Python310\lib\site-packages\anyio\_backe

In [2]:
# import matplotlib.pyplot as plt
# import seaborn as sns


In [1]:
# import seaborn as sns
# from sklearn.metrics import confusion_matrix
# import numpy as np

# # Convert predicted probabilities to binary predictions
# threshold = 0.5  # You can adjust this threshold
# binary_predictions = np.where(predicted_probs >= threshold, 1, 0)

# # Compute confusion matrix
# cm = confusion_matrix(true_labels, binary_predictions)

# # Plot confusion matrix using seaborn
# plt.figure(figsize=(8, 6))
# sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
# plt.title('Confusion Matrix')
# plt.xlabel('Predicted')
# plt.ylabel('True')
# plt.show()

In [3]:
# Assuming you have a trained classifier called "model"
# predicted_probs = model.predict_proba(X)[:, 1]  # Selecting the probabilities for class 1